<a href="https://colab.research.google.com/github/TzurV/TestMe/blob/BERT-based-Decoder/Building_an_E2E_Speech_Recognition_model_using_BERT_4decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an end-to-end Speech Recognition model in PyTorch - [AssemblyAI](https://www.assemblyai.com/)

Adding BERT based Decoder.
Using Private Google drive as Drive for storing.


In [24]:
!python -V

Python 3.6.9


## Speller 

https://stackoverflow.com/questions/13928155/spell-checker-for-python


In [41]:
!pip install autocorrect


     |████████████████████████████████| 1.8MB 2.5MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.1.0-cp36-none-any.whl size=1811979 sha256=ee51049da33a2ff0b32e6d98ba78b647a7fae1d994b4cd20e1b3235533e388f4
  Stored in directory: /root/.cache/pip/wheels/d1/33/69/a42db65d595bb685429c788ab45e11aa2a4b6549298ee38e02
Successfully built autocorrect


In [48]:
from autocorrect import Speller

spell = Speller(lang='en')

print(spell('caaaar'))
print(spell('mussage'))
print(spell('survice'))
print(spell('hte'))

str1 = "i wil make no onjest yus of what i know he replid with frmees"
[(w, spell(w)) for w in str1.split()]


caesar
message
service
the


[('i', 'i'),
 ('wil', 'will'),
 ('make', 'make'),
 ('no', 'no'),
 ('onjest', 'object'),
 ('yus', 'yus'),
 ('of', 'of'),
 ('what', 'what'),
 ('i', 'i'),
 ('know', 'know'),
 ('he', 'he'),
 ('replid', 'replied'),
 ('with', 'with'),
 ('frmees', 'frees')]

In [ ]:
# use inline debuger
from IPython.core.debugger import set_trace
def bad_function(var):
    set_trace()
    return var
bad_function("Mike")

> <ipython-input-33-17a0bbede356>(5)bad_function()
      2 from IPython.core.debugger import set_trace
      3 def bad_function(var):
      4     set_trace()
----> 5     return var
      6 bad_function("Mike")

ipdb> n
--Return--
'Mike'
> <ipython-input-33-17a0bbede356>(5)bad_function()
      2 from IPython.core.debugger import set_trace
      3 def bad_function(var):
      4     set_trace()
----> 5     return var
      6 bad_function("Mike")

ipdb> r


'Mike'

--Return--
None
> <ipython-input-33-17a0bbede356>(6)<module>()
      2 from IPython.core.debugger import set_trace
      3 def bad_function(var):
      4     set_trace()
      5     return var
----> 6 bad_function("Mike")

ipdb> c


## Mount Google Drive

In [1]:
# mount Google Drive
from datetime import datetime
from google.colab import drive
from os import listdir
from os.path import isfile, join
import re 

# mounting is required once every time you start this notebook  
drive.mount('/content/gdrive')
filesRootDirectory = "/content/gdrive/My Drive/Colab_files/CTCmodels/"

# Set to True if you want to check that you have access to google drive
if True:
    # output file 
    # on PC it is located in <Google Drive directory>\Colab_files\CTCmodels
    testOutputFileName = filesRootDirectory + "testOutputFile.txt"
    testOutputFile = open(testOutputFileName,"w") 
    FORMAT = '%Y%m%d%H%M%S'
    TodayDate = datetime.now().strftime(FORMAT)
    print(f" look for {TodayDate} in output file {testOutputFileName}.")
    testOutputFile.write(TodayDate+'\n')
    testOutputFile.close()


Mounted at /content/gdrive
 look for 20201007125602 in output file /content/gdrive/My Drive/Colab_files/CTCmodels/testOutputFile.txt.


## Install Transformenrs (BERT) 
Run basic examples as provided in Transformers (https://huggingface.co/transformers/index.html)

* Masked Language Modeling https://huggingface.co/transformers/task_summary.html#masked-language-modeling

* Causal Language Modeling
https://huggingface.co/transformers/task_summary.html#causal-language-modeling


In [21]:
!pip install transformers

In [23]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelWithLMHead.from_pretrained("distilbert-base-cased")
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."
input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:785: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


AttributeError: ignored

In [18]:
token_logits = model(input)
print(type(token_logits))
print(token_logits[0].size())


<class 'tuple'>
torch.Size([1, 30, 28996])


In [20]:
from transformers import AutoModelWithLMHead, AutoTokenizer, top_k_top_p_filtering
import torch
from torch.nn import functional as F
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")
sequence = f"Hugging Face is based in DUMBO, New York City, and "
input_ids = tokenizer.encode(sequence, return_tensors="pt")
# get logits of last hidden state
next_token_logits = model(input_ids).logits[:, -1, :]
# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)
# sample
probs = F.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)
generated = torch.cat([input_ids, next_token], dim=-1)
resulting_string = tokenizer.decode(generated.tolist()[0])

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:785: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


AttributeError: ignored

## installing the requirements

In [ ]:
!pip install torchaudio==0.4.0 torch==1.4.0 comet-ml==3.0.2

     |████████████████████████████████| 3.1MB 4.8MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 174kB 49.7MB/s 
     |████████████████████████████████| 419kB 49.9MB/s 
     |████████████████████████████████| 204kB 54.5MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34547 sha256=25632b87d4bda2b6c9c9c07fea083ce4b1f53bb2a3744b268dc763ea4b6dd040
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101


## Setting up your data pipeline

In [ ]:
import os

# The core class of Comet.ml is an Experiment, a specific run of a script that 
# generated a result such as training a model on a single set of hyper parameters. 
# An Experiment will automatically log scripts output (stdout/stderr), code, 
# and command line arguments on any script and for the supported libraries will
#  also log hyper parameters, metrics and model configuration.
from comet_ml import Experiment

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio

import numpy as np

def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')

    def int_to_char(self, index):
        if index == 28:
            return '-'
        assert len(self.index_map)>int(index), "Index exceeds map dimension "
        return self.index_map[int(index)]

train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()

def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    # https://pytorch.org/docs/master/generated/torch.nn.utils.rnn.pad_sequence.html#torch-nn-utils-rnn-pad-sequence
    # pad_sequence stacks a list of Tensors along a new dimension, and pads them to equal length. 
    # batch_first (bool, optional) – output will be in B x T x * if True, or in T x B x * otherwise
    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths


def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True, dbg=False):
    if dbg:
        print("# Debugging GreedyDecoder.")
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                if dbg:
                    print(f"({j},{index.item()})")
                decode.append(index.item())
        if dbg and False:
            print(f"({i},text len:{len(decode)})")
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets


## The Model
Base of of Deep Speech 2 with some personal improvements

In [ ]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()
        self.__name__ = 'BidirectionalGRU'

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x


## The Training and Evaluating Script

In [ ]:
def CreateModel(hparams, device):
    ''' create model beased on hparams dictionary'''
    return SpeechRecognitionModel(
            hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
            hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
            ).to(device)

In [ ]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val


def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment):
    model.train()
    data_len = len(train_loader.dataset)
    with experiment.train():
        for batch_idx, _data in enumerate(train_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            experiment.log_metric('loss', loss.item(), step=iter_meter.get())
            experiment.log_metric('learning_rate', scheduler.get_lr(), step=iter_meter.get())

            optimizer.step()
            scheduler.step()
            iter_meter.step()
            if batch_idx % 100 == 0 or batch_idx == data_len:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(spectrograms), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, criterion, epoch, iter_meter, experiment):
    print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with experiment.test():
        with torch.no_grad():
            for i, _data in enumerate(test_loader):
                spectrograms, labels, input_lengths, label_lengths = _data 
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                output = model(spectrograms)  # (batch, time, n_class)
                output = F.log_softmax(output, dim=2)
                output = output.transpose(0, 1) # (time, batch, n_class)

                loss = criterion(output, labels, input_lengths, label_lengths)
                test_loss += loss.item() / len(test_loader)

                #print(f"dbg: calling GreedyDecoder {i}")
                decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
                #print(f"#Ref: {decoded_targets[0]}")
                #print(f"#Rec: {decoded_preds[0]}")
                for j in range(len(decoded_preds)):
                    test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                    test_wer.append(wer(decoded_targets[j], decoded_preds[j]))
                #break

    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)
    experiment.log_metric('test_loss', test_loss, step=iter_meter.get())
    experiment.log_metric('cer', avg_cer, step=iter_meter.get())
    experiment.log_metric('wer', avg_wer, step=iter_meter.get())

    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))


def main(learning_rate=5e-4, batch_size=20, epochs=10,
        train_url="train-clean-100", test_url="test-clean",
        experiment=Experiment(api_key='dummy_key', disabled=True)):

    hparams = {
        "n_cnn_layers": 3,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 29,
        "n_feats": 128,
        "stride":2,
        "dropout": 0.1,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs
    }

    experiment.log_parameters(hparams)

    use_cuda = torch.cuda.is_available()
    torch.manual_seed(7)
    device = torch.device("cuda" if use_cuda else "cpu")

    if not os.path.isdir("./data"):
        os.makedirs("./data")

    train_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=train_url, download=True)
    test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=test_url, download=True)

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'train'),
                                **kwargs)
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'valid'),
                                **kwargs)

    # local creation
    if True:
        model = SpeechRecognitionModel(
            hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
            hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
            ).to(device)
    else:
        model = CreateModel(hparams, device)
        
    print(model)
    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = nn.CTCLoss(blank=28).to(device)
    # torch.optim.lr_scheduler provides several methods to adjust the learning rate based on the number of epochs
    #Sets the learning rate of each parameter group according to the 1cycle learning rate policy. 
    #The 1cycle policy anneals the learning rate from an initial learning rate to 
    # some maximum learning rate and then from that maximum learning rate to some 
    # minimum learning rate much lower than the initial learning rate. 
    # This policy was initially described in the paper Super-Convergence: Very Fast Training of Neural Networks Using Large Learning Rates.
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=hparams['epochs'],
                                            anneal_strategy='linear')
    
    iter_meter = IterMeter()
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment)
        if epoch % 10 == 9 or epoch == epochs:
            test(model, device, test_loader, criterion, epoch, iter_meter, experiment)

    # return the final model
    return model, hparams, device


## Setting up Comet
If you have a comet account, fill in teh api key, project name and experiment name below. You can create an account at [comet.ml](comet.ml).

In [ ]:
comet_api_key = "" # add your api key here
project_name = "speechrecognition"
experiment_name = "speechrecognition-colab"

if comet_api_key:
  experiment = Experiment(api_key=comet_api_key, project_name=project_name, parse_args=False)
  experiment.set_name(experiment_name)
  experiment.display()
else:
  experiment = Experiment(api_key='dummy_key', disabled=True)

## GPU runtime
If you are using a GPU runtime, this will let you know what GPU and how much memory is available. Adjust your batch_size depending on which GPU

In [ ]:
!nvidia-smi

Wed Oct  7 08:24:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## ORIGINAL code sequence 

In [ ]:
def saveModel(model, outputFileName):
    ''' Save NN model into  a file ''' 
    print(f"saving {outputFileName}")
    torch.save(model, outputFileName)

# update save/load as recommended here  
# https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html

def saveModelAsDictionaryWithOptionalExtra(model, outputFileName, extra={}):
    ''' Save NN model into  a file ''' 
    print(f"saving {outputFileName}")

    torch.save({'model':model,
                'model_state_dict': model.state_dict(),
                **extra,
                }, outputFileName)


## Train
this will download the data on first run and may take a while. 

If you have Comet.ml setup, you can start seeing your progress in the comet cell above.

In [ ]:
# Phase I - to make sure things are working
# short training cycle using the the test set for training and 
# evaluation 
learning_rate = 5e-4
batch_size = 10
epochs = 10
libri_train_set = "train-clean-100"
libri_test_set = "test-clean"

learning_rate = 1e-3
batch_size = 10
epochs = 30
libri_train_set = "test-clean"
libri_test_set = "test-clean"

model, hparams, device  = main(learning_rate, batch_size, epochs, libri_train_set, libri_test_set, experiment)

modelFileName = filesRootDirectory + "PhaseImodel_20Iter_test.pt"
saveModelAsDictionaryWithOptionalExtra(model, modelFileName, hparams)



SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type SpeechRecognitionModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ResidualCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNNLayerNorm. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BidirectionalGRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be check

# Just recognition
* load stored model
* run recognition


In [ ]:
if not os.path.isdir("./data"):
        os.makedirs("./data")
        
test_url="test-clean"
test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=test_url, download=True)


In [ ]:
print(type(test_dataset))

# https://pytorch.org/docs/stable/data.html

use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

print(use_cuda)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=5, # originaly 20
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'valid'),
                                **kwargs)

print(type(test_loader))

<class 'torchaudio.datasets.librispeech.LIBRISPEECH'>
True
<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:


#print(model.state_dict())
#hparams = {
#        "n_cnn_layers": 3,
#        "n_rnn_layers": 5,
#        "rnn_dim": 512,
#        "n_class": 29,
#        "n_feats": 128,
#        "stride":2,
#        "dropout": 0.1,}
#saveModelAsDictionaryWithOptionalExtra(model, modelFileName, extra=hparams)

# test loading
modelFileName = filesRootDirectory + "PhaseImodel_20Iter_test.pt"

checkpoint = torch.load(modelFileName)
print(checkpoint.keys())

model1 = CreateModel(checkpoint, device)
model1.load_state_dict(checkpoint['model_state_dict'])
model1.eval()

criterion1 = nn.CTCLoss(blank=28).to(device)
iter_meter = IterMeter()
test(model1, device, test_loader, criterion1, 0, iter_meter, experiment=Experiment(api_key='dummy_key', disabled=True))

dict_keys(['model', 'model_state_dict', 'n_cnn_layers', 'n_rnn_layers', 'rnn_dim', 'n_class', 'n_feats', 'stride', 'dropout', 'learning_rate', 'batch_size', 'epochs'])

evaluating...
Test set: Average loss: 0.5921, Average CER: 0.184106 Average WER: 0.5713



In [ ]:
# Understand GreedyDecoder output
# stops for debugging on errors (?)
#%debug

#del decoded_preds, decoded_targets
with torch.no_grad():
    # groups side depends on batch_size=5
    for i, _data in enumerate(test_loader):
        spectrograms, labels, input_lengths, label_lengths = _data 
        spectrograms, labels = spectrograms.to(device), labels.to(device)

        output = model1(spectrograms)  # (batch, time, n_class)
        output = F.log_softmax(output, dim=2)
        output = output.transpose(0, 1) # (time, batch, n_class)

        print(f"#Inference output #dim={output.dim()}, {output.size()}, {output.transpose(0, 1).size()}")
        print(f"# reference text: {text_transform.int_to_text(labels[i][:label_lengths[i]].tolist())}")
        #print(labels[i][:label_lengths[i]])
        
        outputCopy = output.detach().clone()
        for jj in range(label_lengths[i]):
            outputArgmax1 = int(torch.argmax(outputCopy[jj][0][:]))
            outputMax1    = float(outputCopy[jj][0][outputArgmax1])
            outputCopy[jj][0][outputArgmax1] = -100.0
            outputArgmax2 = int(torch.argmax(outputCopy[jj][0][:]))
            outputMax2    = float(outputCopy[jj][0][outputArgmax2])

            print(f"{outputArgmax1:2d} {text_transform.int_to_char(outputArgmax1):2} {outputMax1:4.2f}; \
                    {outputArgmax2:2d} {text_transform.int_to_char(outputArgmax2):2} {outputMax2:4.2f}")
            

        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        print(type(decoded_preds), len(decoded_preds), type(decoded_preds[0]))


        print(f"# predicted text: !{decoded_preds}!")


        break

#Inference output #dim=3, torch.Size([430, 5, 29]), torch.Size([5, 430, 29])
# reference text: i will make no unjust use of what i know he replied with firmness i believe you my lord
28 -  -0.00;                     10 i  -9.96
28 -  -0.00;                     10 i  -10.07
28 -  -0.00;                     10 i  -10.31
28 -  -0.00;                     10 i  -10.28
28 -  -0.00;                     10 i  -9.98
28 -  -0.00;                     10 i  -9.87
28 -  -0.00;                     10 i  -10.28
28 -  -0.00;                     10 i  -9.92
28 -  -0.00;                     10 i  -9.83
28 -  -0.00;                     10 i  -9.78
28 -  -0.00;                     10 i  -9.88
28 -  -0.00;                     21 t  -10.49
28 -  -0.00;                     21 t  -7.15
28 -  -0.00;                     10 i  -6.39
10 i  -0.02;                      6 e  -4.50
28 -  -0.02;                     26 y  -4.97
28 -  -0.11;                      1    -2.66
 1    -0.02;                      6 e  -5.45
 1

## Original Main

In [ ]:
learning_rate = 5e-4
batch_size = 10
epochs = 10
libri_train_set = "train-clean-100"
libri_test_set = "test-clean"

main(learning_rate, batch_size, epochs, libri_train_set, libri_test_set, experiment)

In [ ]:
experiment.end()

# My learning code 


In [ ]:
if not os.path.isdir("./data"):
        os.makedirs("./data")
        
test_url="test-clean"
test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=test_url, download=True)


In [ ]:
print(type(test_dataset))

# https://pytorch.org/docs/stable/data.html

use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

print(use_cuda)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=5, # originaly 20
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'valid'),
                                **kwargs)

print(type(test_loader))

<class 'torchaudio.datasets.librispeech.LIBRISPEECH'>
True
<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:
import matplotlib.pyplot as plt

local_audio_transforms = nn.Sequential(
    # https://pytorch.org/audio/transforms.html#melspectrogram
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)#,
    # https://pytorch.org/audio/transforms.html#frequencymasking
    #torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    # https://pytorch.org/audio/transforms.html#timemasking
    #torchaudio.transforms.TimeMasking(time_mask_param=100)
)

spectrograms = []
labels = []
input_lengths = []
label_lengths = []

inx = 0
for (waveform, sample_rate, utterance, B, C, D) in test_dataset:
    inx += 1
    print(type(sample_rate))
    print(type(B))
    print(type(C))
    print(type(D))
    print(type(waveform))
    print(type(utterance))
    print(utterance)
    print(f"sample_rate={sample_rate} B={B} C={C} D={D}")
    
    print(type(waveform))
    print("Shape of waveform: {}".format(waveform.size()))
    print("Sample rate of waveform: {}".format(sample_rate))

    if inx == 10:
        # define 2 sub plots
        fig, axs = plt.subplots(2)
        #fig.suptitle('Vertically stacked subplots')
        #axs[0].plot(x, y)
        #axs[1].plot(x, -y)

        #plt.figure()
        axs[0].plot(waveform.t().numpy())  

        spec = local_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        print("type(spec) is ", type(spec)) 
        print("Shape of spec: {}".format(spec.size()))
        #    axs[1].imshow(spec.log2().detach().numpy(), cmap='gray')
        axs[1].imshow(torch.transpose(spec, 0, 1).log2().numpy(), cmap='gray')


    # the 4 elements of the test_loader are defined by  def data_processing(..)
    #           return  spectrograms, labels, input_lengths, label_lengths
    spectrograms.append(spec)
    print("#>", len(spectrograms))
    label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
    print(utterance.lower())
    print(text_transform.text_to_int(utterance.lower()))
    #labels.append(label)
    input_lengths.append(spec.shape[0]//2)
    print(spec.shape)
    print(spec.shape[0]//2)
    #label_lengths.append(len(label))
    print(len(label))


    if inx == 5:
        break



In [ ]:
# https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

learning_rate=5e-4
batch_size=20
epochs=10
hparams = {
        "n_cnn_layers": 3,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 29,
        "n_feats": 128,
        "stride":2,
        "dropout": 0.1,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs
    }

model = SpeechRecognitionModel(
    hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
    hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
    ).to(device)

print(model)
print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

oneWhat = next(iter(test_loader))
#oneWhat = iter(test_loader)
print(type(oneWhat))
print(len(oneWhat))
print([type(x) for x in oneWhat])
print(oneWhat[0].size())
print(oneWhat[1].size())
print(oneWhat[2])
print(sum(oneWhat[2]))
print(oneWhat[3])
print(sum(oneWhat[3]))

#writer.add_graph(model, oneWhat)
#writer.close()